In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies=pd.read_csv('/content/my csv/tmdb_5000_movies.csv')
credit=pd.read_csv('/content/my csv/tmdb_5000_credits.csv', engine='python')

FileNotFoundError: [Errno 2] No such file or directory: '/content/my csv/tmdb_5000_movies.csv'

In [ ]:
credit.head(1)

In [ ]:
movies=movies.merge(credit,on='title')

In [ ]:
movies.head(1)

In [ ]:
movie=movies[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movie.head(1)

In [ ]:
movie.isnull().sum()

In [ ]:
movie.dropna(inplace=True)

In [ ]:
movie.isnull().sum()

In [ ]:
movie.shape

In [ ]:
movie.duplicated().sum()

NameError: name 'movie' is not defined

In [ ]:
movie.iloc[0].genres

In [ ]:
import ast

In [ ]:
def convert(obj):
  l=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter!=3:
      l.append(i['name'])
      counter+=1
    else:
      break
  return l

In [ ]:
def fetch_director(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [ ]:
movie['genres']=movie['genres'].apply(convert)

In [ ]:
movie['keywords']=movie['keywords'].apply(convert)

In [ ]:
movie['cast']=movie['cast'].apply(convert)

In [ ]:
movie['crew']=movie['crew'].apply(fetch_director)

In [ ]:
movie.head(5)

In [ ]:
def overview(obj):
  l=[]
  l.append(obj)
  return l

In [ ]:
movie['overview']=movie['overview'].apply(overview)

In [ ]:
movie.head()

In [ ]:
movie['genres']=movie['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movie['keywords']=movie['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movie['cast']=movie['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movie['crew']=movie['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [ ]:
movie.head(1)

NameError: name 'movie' is not defined

In [ ]:
movie['tags']=movie['overview']+movie['genres']+movie['keywords']+movie['cast']+movie["crew"]

In [ ]:
movie.head(1)

In [ ]:
new_df=movie[['id','title','tags']]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head(1)

In [ ]:
import nltk
ps=nltk.stem.porter.PorterStemmer()

In [ ]:
def stem(text):
  y=[]
  for  i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors.shape

In [ ]:
cv.get_feature_names_out()

NameError: name 'cv' is not defined

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
similarity[1]

In [ ]:
def recommend(movie):
  movie_index =new_df[new_df['title']== movie].index[0]
  distances= similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]].title)


In [ ]:
recommend("Batman")

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))